# Week 1:
## Monday May 20, 2019:
* first day!
* setup and luncheon
* researched Roary

**Brief project history:**

Thomas Pedersen was a UAMS PhD student who originally created [FindMyFriends](https://github.com/thomasp85/FindMyFriends) to be able to rapidly construct the pangenome (core and accessory genes) across multliple bacterial genomes quickly and accurately. The program first uses an algorithm called [CD-Hit](https://www.ncbi.nlm.nih.gov/pubmed/11294794) to quickly group the genes into broad groups, and then the gene clusters are split into smaller clusters based on k-mer similarity and neighboring sequences (to separate orthologues (difference from speciation) from paralogues (difference from duplication)). Once the genes are grouped, it can be determined to what degree certain genes are shared between the genomes that were fed to the algorithm. 

While Thomas was working on FindMyFriends, a separate pan-genome construction algorithm was published called [Roary](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4817141/). Roary accomplished the same task faster, so Thomas decided to use design elements like CD-Hit from Roary to improve the speed of FindMyFriends. Once Thomas graduated with his PhD, FindMyFriends was left in an unfinished state. *Consequently, the goal of this project is to replace the CD-Hit pre-clustering step with a kmer-counting algorithm to try to restore functionality and accuracy to FindMyFriends.*

**Research Notes:**

Programs like PanOCT and PGAP run all-against-all BLASTs across the input genomes, but this scales quadratically with computation time and memory usage, so it becomes impractical to use these programs for large amounts of data. Roary (and FindMyFriends) instead use kmer counting. If two sequences are similar, they must exactly share a certain number of short sequences (kmers). In the original CD-Hit paper, they provide an example: "[T]wo sequences having 85% identical residues over a 100-residue window will have at least 70 identical dipeptides,55 tripeptides and 25 pentapeptides" (Li et al., 2001). Sequences can then be quickly compared and grouped so that a BLAST does not have to be run for all of them. The problem with the CD-Hit algorithm (as used in the latest version of FindMyFriends) is that once it groups the genomes based on kmer counts, it randomly chooses a representative sample from each group to be BLASTed against other groups. This eliminates reproducible results when running the algorithm. 

**References**

Page AJ, Cummins CA, Hunt M, et al. Roary: rapid large-scale prokaryote pan genome analysis. Bioinformatics. 2015;31(22):3691–3693. doi:10.1093/bioinformatics/btv421

Weizhong Li, Lukasz Jaroszewski, Adam Godzik. Clustering of highly homologous sequences to reduce the size of large protein databases. Bioinformatics. 2001;17(3):282–283, https://doi.org/10.1093/bioinformatics/17.3.282
***

## Tuesday May 21, 2019:
* got files from Kaleb
* downloaded prodigal, RStudio
* generated heatmap of clusters of genes from a single genome
* downloaded 22 genomes (incorrectly)

**Generating .faa files**

Create a text file containing assemblies that are to be downloaded with one assembly identifier on each line. Go to [Batch Entrez](https://www.ncbi.nlm.nih.gov/sites/batchentrez) and upload the file to retrieve the .fna files from GenBank (not RefSeq!) for each assembly. Run prodigal to convert the .fna files to .faa files:

**prodigal -i inputfilename.fna -a outputfilename.faa**

For the single genome heatmap, I used the GCA_00369165.2 chromosome assembly from NCBI. 
For the 25 genomes, I used:
* GCA_000092525.1
* GCA_000006925.2
* GCA_000006665.1
* GCA_000008865.2
* GCA_000019645.1
* GCA_000176695.2
* GCA_000012005.1
* GCA_000176575.2
* GCA_000350785.1
* GCA_000176655.2
* GCA_000026325.2
* GCA_000091005.1
* GCA_000010485.1
* GCA_000007445.1
* GCA_000690815.1
* GCA_000734955.1
* GCA_000613265.1
* GCA_000333215.1
* GCA_000714595.1
* GCA_000167875.1
* GCA_000012025.1
* GCA_000005845.2
* GCA_000010245.1
* GCA_002007705.1
* GCA_900092615.1

**Generating the heatmap**

In [ ]:
library("ape")
library("kmer")
library("MASS")
setwd("C:/Users/Matthew/Documents/UAMS SURF/K-mer testing/FAA files")

aminoAcids <- read.FASTA("GCF_000005845.2_ASM584v2_genomic.faa", type = "AA")
# create a distance matrix using 4-mers from each protein from the single genome
aminoAcidsDist <- kdistance(aminoAcids, k = 4)
# roughly group the proteins into a dendrogram
aminoAcidsTree <- as.dendrogram(hclust(aminoAcidsDist), "average")
plot(heatmap(as.matrix(aminoAcidsDist), Rowv=aminoAcidsTree, Colv = "Rowv", scale = "none"))
# optionally print out the kmer distance matrix
write.matrix(aminoAcidsDist, file = "file_name", sep= " ")

My run using the GCA_00369165.2 chromosome with k=4 generated the heatmap below. You can see that where the dendrogram goes down many layers, the heatmap area lightens. The sensitivity of the heatmap can be changed to show larger, clearer clusters, but this was just a test to see if a heatmap could be generated.
![heatmap](images/AA4merHeatMap_GCA_003697165.2.png)

## Wednesday May 22, 2019:
* setup jupyter notebook
* download 25 genomes from GenBank (correctly this time)
* work on getting top 5 9-mers for each protein in a single genome

Today consisted of a lot of setup and troubleshooting with RStudio while trying to working on the 9-mers. I was trying to create a large matrix of kmer counts for the proteins of a genome using the kmer R package, but my computer could not do it for all of the proteins at once. Code:


In [ ]:
library("ape")
library("kmer")
library("MASS")
setwd("C:/Users/Matthew/Documents/UAMS SURF/K-mer testing/FAA files")
proteins <- read.FASTA("GCA_000005845.2_ASM584v2.faa", type = "AA")
kmerCounts <- kcount(proteins[0:200], k = 9)

I was able to get the kcount to work for 200 of the proteins, and it produced a 200 x 10077696 matrix. The rows were the 200 protein sequences and there were n^k columns, where n is the alphabet size (n = 6 for reduced amino acid classes: AGPST, C, DENQ, FWY, HKR, ILMV) and k is the kmer length (k = 9 for this example). If I access the matrix using **kmerCounts[1,1:20]**, I am able to see the kmer counts for specific kmers for the first sequence. 

![kmerCounts](images/kmerCount.JPG)


Tomorrow, the goal is to sort the columns for each row to get the 5 top occuring kmers for each sequence.

## Thursday May 23, 2019:
Today the goal was to get kmer count data from multiple genomes all at once to generate a heatmap. Kaleb modified my R program to read in all of the genomes and output the kmer count matrices to .csv files. I can also remove the amino acid compression in the code by setting "compress" to "FALSE" to ensure that the sequence uniqueness is conserved, but we kept it compressed for testing. Not compressing does increase the size of the kmer matrix though, because there are 20^9 possibilities for 9mers. Code:

library("ape")
library("kmer")
library("MASS")
setwd("/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/FAA files")

#genomes that were previously downloaded through genbank and processed to .faa files using prodigal
file_list <-c("GCA_000005845.2_ASM584v2","GCA_000006665.1_ASM666v1","GCA_000006925.2_ASM692v2","GCA_000007445.1_ASM744v1","GCA_000008865.2_ASM886v2","GCA_000010245.1_ASM1024v1","GCA_000010485.1_ASM1048v1","GCA_000012005.1_ASM1200v1","GCA_000012025.1_ASM1202v1","GCA_000019645.1_ASM1964v1","GCA_000026325.2_ASM2632v2","GCA_000091005.1_ASM9100v1","GCA_000092525.1_ASM9252v1","GCA_000167875.1_ASM16787v1","GCA_000176575.2_ASM17657v2","GCA_000176655.2_ASM17665v2","GCA_000176695.2_ASM17669v2","GCA_000333215.1_ASM33321v1","GCA_000350785.1_Esch_coli_KTE21_V1","GCA_000613265.1_ASM61326v1","GCA_000690815.1_ASM69081v1","GCA_000714595.1_ASM71459v1","GCA_000734955.1_ASM73495v1","GCA_002007705.1_ASM200770v1","GCA_003697165.2","GCA_900092615.1_PRJEB14041")

for (file in file_list)
{
  file_path <- paste(file,'.faa',sep = '')
  out_path <- paste(file,'_kmer_matrix.csv',sep = '')
  proteins <- read.FASTA(file_path, type = "AA")
  print(paste('Genome ',file,' proteins are loaded'))
  kmerCounts <- kcount(proteins, k = 4, compress = TRUE)
  print(paste('Genome ',file, ' kmerCounts is finished'))
  write.csv(as.matrix(kmerCounts), file = out_path)
  print(paste('Genome ', file, ' matrix is output'))
  print(paste('Genome ',file,' is complete', sep = ''))
}

We first tried running the above code with k=9, but the larger set of kmers made the data too large for my computer to handle. To proceed, we ran the code with k=4 and generated .csv files of kmer counts for all of the genomes. Next, Kaleb wrote a python program that opened the .csv files, cleaned up the protein names, and placed the data into a pandas (python package) dataframe. Next, the dataframe is transposed (kmers and rows, genomes are columns), kmers with a count of 0 are removed from the column of interest, the column is sorted, (optionally, only the top 10 kmers are retained), and then the column is added to a large dataframe. 

import glob, pandas as pd
data_folder = '/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/FAA files/'
#create a list of files that match regular expression
file_list = glob.glob(data_folder + '*kmer_k4_matrix*')
dicty = dict()
for file in file_list:
    df = pd.read_csv(file)
    #gets protein_ids
    protein_id = list(df['Unnamed: 0'])
    #cleans protein header to only get protein_id
    protein_id = [x.split(' ')[0] for x in protein_id]
    df['protein_id'] = protein_id
    #move from data column to index column
    df.set_index('protein_id',inplace = True)
    #delete protein header list
    del df['Unnamed: 0']
    
    #transpose so that kmers are rows and genomes are columns
    df = df.T
    
    for col in list(df.columns):
        #get whole data frame where a certain criteria is met on a column
        #on the column, go through the list and only get the rows that do not equal zero
        loop = df[df[col] != 0]
        loop = loop[col]
        loop = loop.sort_values(ascending = False)
        #loop = loop[0:10]
        dicty[col] = loop

#when merging series, if row is not present, then fill value for that row with N/A
df_test = pd.DataFrame(dicty)

#replace N/A with 0
df2 = df_test.fillna(0)
df2.to_csv('/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/FAA files/k4_kmer_all_table.csv')

We then took the large .csv file and tried to generate a heatmap, but we encountered an error late in the program. Instead of waiting for another heatmap, Kaleb decided to generate a dendrogram of the data using the R code below.

In [ ]:
in_data <- as.matrix(read.csv(file = ".csv file", sep = ',', header = T, row.names = 1))

testy <- as.dist(1-cor(in_data))
# hierarchical clustering
hc <- hclust(testy, mehtod = "average")


# plot without sensitivity and color
plot(hc)

# change h to have more or less grouping sensitivity
mcl <- cutree(hc, h=0.8)
# color groups in the dendrogram
ColorDendrogram(hc, y = mcl)

The dendrogram will be generated overnight. Plan for tomorrow: 
* generate .csv for 5mers 
* try using the cluster() method from the R kmer package to cluster the data instead of the heirarchical clustering
    * give cluster a multiFASTA file so that it is converted into an AAbin file
    * make multiFASTA by concatenating .faa files: `ls *.faa | xargs cat >> multifasta.faa`
    * cluster() will go around .csv files, etc. 
    
Basically, the kmer counting is taking a long time and does not scale well, so the kmer package cluster method is our next plan.